In [1]:
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from typing import List, Optional, Dict, Tuple
from langchain_core.prompts import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_aws import ChatBedrock
from langchain_core.runnables.history import RunnableWithMessageHistory
import time
from botocore.exceptions import ClientError
import asyncio
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory


In [2]:
def get_dynamodb_credentials():
    """Get DynamoDB credentials from AWS Secrets Manager"""
    
    # Initialize Secrets Manager client
    secrets_client = boto3.client('secretsmanager')
    
    try:
        # Get the secret value
        response = secrets_client.get_secret_value(SecretId='dynamodb-credentials')
        
        # Parse the JSON string
        secret_data = json.loads(response['SecretString'])
        
        return secret_data
        
    except Exception as e:
        print(f"Error retrieving secret: {e}")
        raise

In [3]:
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [4]:
class DynamoDBHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, table_name: str = "conversations-table"):
        self.session_id = session_id
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
    
    @property
    def messages(self) -> list[BaseMessage]:
        """DynamoDB에서 메시지 로드"""
        try:
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id),
                ScanIndexForward=True,  
                Limit=10,
            )
            
            messages = []
            for item in response['Items']:
                if item['role'] == 'human':
                    messages.append(HumanMessage(content=item['message']))
                elif item['role'] == 'ai':
                    messages.append(AIMessage(content=item['message']))
            
            return messages
        except Exception as e:
            print(f"메시지 로드 실패: {e}")
            return []
    
    def add_message(self, message: BaseMessage) -> None:
        """새 메시지를 DynamoDB에 저장"""
        sequence = int(time.time() * 1000)  # millisecond timestamp
        
        if isinstance(message, HumanMessage):
            role = 'human'
        elif isinstance(message, AIMessage):
            role = 'ai'
        else:
            return
        
        self.table.put_item(
            Item={
                'session_id': self.session_id,
                'sequence': sequence,
                'role': role,
                'message': message.content,
                'timestamp': datetime.now().isoformat()
            }
        )
    
    def clear(self) -> None:
        """대화 기록 삭제"""
        try:
            # Query all items for this session
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id)
            )
            
            # Delete each item
            for item in response['Items']:
                self.table.delete_item(
                    Key={
                        'session_id': item['session_id'],
                        'sequence': item['sequence']
                    }
                )
        except Exception as e:
            print(f"메시지 삭제 실패: {e}")

In [5]:

credentials = get_dynamodb_credentials()
table_name = credentials['table_name']
region = credentials['region']
table_arn = credentials['table_arn']

# Use in DynamoDB client
dynamodb = boto3.resource('dynamodb', region_name=region)
table = dynamodb.Table(table_name)

# 전역 변수
agent = None

In [6]:
async def extract_text(payload):
    """텍스트 추출 AgentCore Runtime 엔트리포인트"""
    global agent
    global table_name
    
    if agent is None:
        yield {"type": "status", "message": "🚀 LLM 초기화 중..."}
        agent = AdvancedLLM()
    
    # payload에서 입력 데이터 추출
    user_input = payload.get("input_data", "태양의 온도에 대해 말해줘")
    session_id = payload.get("seesion_id", "test-session")  # Fixed typo

    # 체인 구성
    template_dynamo = """The following is a friendly conversation between a human and an AI. 
                        The AI is talkative and provides lots of specific details from its context. 
                        
                        Relevant Information:
                        
                        {chat_history}
                        
                        Conversation:
                        Human: {question}
                        AI:"""
    
    prompt_dynamo = PromptTemplate(
        input_variables=["chat_history", "question"], 
        template=template_dynamo
    )

    # Create chain with DynamoDB history
    chain = prompt_dynamo | agent.llm | StrOutputParser()
    
    chain_dynamo = RunnableWithMessageHistory(
        chain,
        get_session_history=lambda session_id: DynamoDBHistory(session_id, table_name),
        input_messages_key="question",
        history_messages_key="chat_history"
    )
    
    yield {"type": "status", "message": "🔥 응답 생성 중..."}
    
    collected_text = []
    try:
        # Use chain_dynamo and pass proper input
        async for event in chain_dynamo.astream_events(
            {"question": user_input},
            config={"configurable": {"session_id": session_id}}
        ):
            if event["event"] == "on_chat_model_stream":
                chunk = event["data"]["chunk"]
                if hasattr(chunk, 'content') and chunk.content:
                    if isinstance(chunk.content, str):
                        collected_text.append(chunk.content)
                        yield {"type": "stream", "content": chunk.content}
                                
    except Exception as e:
        yield {"type": "error", "message": f"❌ 스트리밍 실패: {e}"}
    
    # 최종 결과
    final_text = "".join(collected_text)
    yield {"type": "final", "content": final_text}

In [7]:
# Test payload
test_payload = {
    "input_data": "내가 생각하는 상사병은 무엇인가요?",
    "seesion_id": "test-session-001"
}

print("🚀 Starting chat...")

# Run the extract_text function
async for result in extract_text(test_payload):
    if result["type"] == "status":
        print(f"📊 {result['message']}")
    elif result["type"] == "stream":
        print(result["content"], end="", flush=True)
    elif result["type"] == "error":
        print(f"\n❌ {result['message']}")
    elif result["type"] == "final":
        print(f"\n\n✅ Final result: {len(result['content'])} characters")



🚀 Starting chat...
📊 🚀 LLM 초기화 중...
📊 🔥 응답 생성 중...
안녕하세요! 죄송하지만 제가 현재 가지고 있는 정보로는 당신이 개인적으로 상사병에 대해 어떻게 생각하고 계시는지 알 수 없습니다.

상사병에 대한 일반적인 관점들을 말씀드리자면:

- **의학적 관점**: 상사병은 전통적으로 한의학에서 사용되던 용어로, 깊은 그리움이나 사랑의 감정이 신체적 증상으로 나타나는 것을 의미했습니다
- **심리학적 관점**: 현대에는 강한 감정적 스트레스나 일방적인 사랑으로 인한 심리적, 신체적 증상들로 이해됩니다
- **문화적 관점**: 문학이나 드라마에서는 로맨틱한 소재로 다뤄지기도 합니다

혹시 상사병에 대한 당신만의 생각이나 경험이 있으시다면 더 자세히 말씀해 주시겠어요? 그러면 더 구체적이고 의미 있는 대화를 나눌 수 있을 것 같습니다!

✅ Final result: 380 characters


In [8]:
test_payload = {
    "input_data": "내가 생각하는 상사병은 상사로 인한 스트레스를 받는 병을 말하는겁니다",
    "seesion_id": "test-session-001"
}

print("🚀 Starting chat...")

# Run the extract_text function
async for result in extract_text(test_payload):
    if result["type"] == "status":
        print(f"📊 {result['message']}")
    elif result["type"] == "stream":
        print(result["content"], end="", flush=True)
    elif result["type"] == "error":
        print(f"\n❌ {result['message']}")
    elif result["type"] == "final":
        print(f"\n\n✅ Final result: {len(result['content'])} characters")



🚀 Starting chat...
📊 🔥 응답 생성 중...
아, 그렇군요! 정말 재미있는 관점이네요! 😄 

당신이 말씀하신 "상사병"은 직장에서 상사로 인해 받는 스트레스와 그로 인한 신체적, 정신적 증상들을 의미하는 거군요. 이는 전통적인 상사병(相思病, 그리움의 병)과는 완전히 다른 현대적 해석이네요!

실제로 직장에서 상사와의 관계로 인한 스트레스는 정말 심각한 문제입니다:

**증상들:**
- 월요병이 심해지거나 출근 거부감
- 불면증, 두통, 소화불량
- 불안감, 우울감 증가
- 혈압 상승, 면역력 저하

**원인들:**
- 과도한 업무 압박이나 불합리한 요구
- 소통 부재나 일방적인 지시
- 인격적 모독이나 부당한 대우
- 업무 스타일의 극심한 차이

이런 "상사병"을 겪고 계신 건가요? 아니면 주변에서 이런 상황을 자주 보셔서 이렇게 정의하게 되신 건지 궁금합니다. 정말 현실적이고 공감되는 표현이에요!

✅ Final result: 429 characters


In [9]:
# Test payload
test_payload = {
    "input_data": "내가 생각하는 상사병은 무엇인가요?",
    "seesion_id": "test-session-001"
}

print("🚀 Starting chat...")

# Run the extract_text function
async for result in extract_text(test_payload):
    if result["type"] == "status":
        print(f"📊 {result['message']}")
    elif result["type"] == "stream":
        print(result["content"], end="", flush=True)
    elif result["type"] == "error":
        print(f"\n❌ {result['message']}")
    elif result["type"] == "final":
        print(f"\n\n✅ Final result: {len(result['content'])} characters")



🚀 Starting chat...
📊 🔥 응답 생성 중...
앞서 대화에서 당신이 말씀해주신 바로는, 당신이 생각하는 상사병은 **"상사로 인한 스트레스를 받는 병"**이라고 하셨죠! 

정말 창의적이고 현실적인 해석이었어요. 전통적인 상사병(相思病, 그리움의 병)이 아니라, 직장에서 상사 때문에 받는 스트레스로 인해 생기는 각종 신체적, 정신적 증상들을 의미한다고 설명해주셨습니다.

이 "상사병"의 개념이 정말 많은 직장인들에게 공감될 것 같아요. 실제로 상사와의 관계는 직장 생활의 만족도와 건강에 엄청난 영향을 미치니까요.

혹시 이 개념을 생각하게 된 특별한 계기가 있으셨나요? 아니면 이런 상사병을 예방하거나 치료하는 방법에 대해서도 나름의 생각이 있으신지 궁금합니다! 😊

✅ Final result: 349 characters
